In [ ]:
gcloud auth list

In [ ]:
gcloud config list project

In [ ]:
PROJECT_ID=$(gcloud config get-value project)
export BUCKET_NAME="${PROJECT_ID}-bucket"
export REGION=us-east4

##Task 1. Create a Pub/Sub topic

In [ ]:
export TOPIC_ID=mypubsub

In [ ]:
gcloud services disable dataflow.googleapis.com
gcloud services enable dataflow.googleapis.com

##Task 3. Create a Cloud Storage bucket

In [ ]:
gsutil mb gs://$BUCKET_NAME

In [ ]:
gcloud pubsub topics create $TOPIC_ID

##Task 2. Create a Cloud Scheduler job

In [ ]:
export MESSAGE="Hello World!"

In [ ]:
gcloud app create --region=$REGION

In [ ]:
gcloud scheduler jobs create pubsub loli --schedule="* * * * *" \
    --topic=$TOPIC_ID --message-body="$MESSAGE"

In [ ]:
gcloud scheduler jobs run loli

##Task 4. Run a Dataflow pipeline to stream data from a Pub/Sub topic to Cloud Storage

In [ ]:
cat > run_pubsub_to_gcs_loli.sh <<EOF_CP
#!/bin/bash

# Set environment variables
export PROJECT_ID=$PROJECT_ID
export REGION=$REGION
export TOPIC_ID=$TOPIC_ID
export BUCKET_NAME=$BUCKET_NAME

# Clone the repository and navigate to the required directory
git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
cd python-docs-samples/pubsub/streaming-analytics

# Install dependencies
pip install -U -r requirements.txt

# Run the Python script with parameters
python PubSubToGCS.py \
  --project=$PROJECT_ID \
  --region=$REGION \
  --input_topic=projects/$PROJECT_ID/topics/$TOPIC_ID \
  --output_path=gs://$BUCKET_NAME/samples/output \
  --runner=DataflowRunner \
  --window_size=2 \
  --num_shards=2 \
  --temp_location=gs://$BUCKET_NAME/temp
EOF_CP

In [ ]:
chmod +x run_pubsub_to_gcs_loli.sh

In [ ]:
gcloud scheduler jobs run loli

In [ ]:
docker run -it \
  -e DEVSHELL_PROJECT_ID=$DEVSHELL_PROJECT_ID \
  -v "$(pwd)/run_pubsub_to_gcs_loli.sh:/run_pubsub_to_gcs_loli.sh" \
  python:3.7 \
  /bin/bash -c "/run_pubsub_to_gcs_loli.sh"